In [6]:
import torch
from torch import Tensor
import numpy as np

In [7]:
class Activation_functional (intput):
    
    def __init__(self,input,act):
        self.result = Tensor.new(input)
        self.kind   = act   # Relu, Sig, Relu' , sig'
        
    def __getitem__(self, index):
        return self.train_aug[index], self.train_target[index]
    
    def __len__(self):
        return len(self.train_target)
    
    
        
    def dRefu (self,input):
        result = 1-(input<0)
        
    def dSigmoid (self,input):
        result = 1-torch.pow(torch.tanh(input),2)
        
    
        
    def dTanh(self,input):
        result = 1-torch.tanh(x)*torch.tanh(x)
    
    return result

SyntaxError: 'return' outside function (<ipython-input-7-ad04dc6900f7>, line 26)

In [ ]:
class Seqeuntial (arch_list):
    
    def __init__(self,input,act):
        self.result = Tensor.new(input)
        self.kind   = act   # Relu, Sig, Relu' , sig'
        
    def __getitem__(self, index):
        return self.train_aug[index], self.train_target[index]
    
    def __len__(self):
        return len(self.train_target)
    
    def make_net (self,arch_list):
        nb = len(self.arch_list)
        
        for i in range (nb):
            check_name (arch_list[i])

# Auxiliary Classes

In [117]:
class Weight:
    """
    Initializes the synaptic weights
    
    inputs:
        layer_nb    :  the number of layer
        dim_in      :  the input dimension of fully connected layer
        dim_out     :  the output dimension of fully connected layer
        
    returns:
        Nothing
    """
    
    def __init__(self,layer_nb, dim_in, dim_out):
        self.shape = (dim_in , dim_out)
        self.name = layer_nb
        self.data = Tensor (dim_in,dim_out).normal_()

# it is a module in the net. it calls some specific parameter...   


class Linear:
    """
    An class that contains objects which only store layar's in/out connections dimension
    
    inputs:
        dim_in      :  the input dimension of fully connected layer
        dim_out     :  the output dimension of fully connected layer
        
    returns:
        Nothing 
    """
    
    def __init__(self,dim_in, dim_out):
        self.input = dim_in
        self.output = dim_out          

        
def Activation(code,input):
        
    result = torch.Tensor.new(input.shape)
    
    if code ==0:
        result = input - (input<0).float()*input
    elif code ==1:
        result = torch.tanh(input)
    elif code ==2:
        result = 1.0/(1 + torch.exp(-input))
    else: raise UnknownCodeForActivation
    return result 
"""
    A class that specify the needed activation with respect to the following code conversion
        0: Relu(x)
        1: Tanh(x)
        2: Sigmoid(x)
    
    This class works as functional package of pytorch
    
    inputs:
        code        :  the code for each activation (0,1,2)
        input_tensor:  the input tensor
        
    returns:
        result      :  the output of requested activation function with the same shape as input tensor
    
    
    def __init__(self, code, input):
        self.code = code
        
        self.result = torch.Tensor.new(input.shape)
    
    def Relu (self,input):
        result = input - (input<0);
        
    def Tanh (self,input): #WARNING!!!
        result = torch.tanh(input)
        
    def Sigmoid (self,input):
        result = 1.0/(1 + torch.exp(-input))

    
    def choose_activation (self, code,input):
        if self.code ==0:
            self.result = self.Relu(input)
        elif self.code ==1:
            self.result = self.Tanh(input)
        elif self.code ==2:
            self.result = self.Sigmoid(input)
        else: raise UnknownCodeForActivation
        return self.result 
        """

'\n    A class that specify the needed activation with respect to the following code conversion\n        0: Relu(x)\n        1: Tanh(x)\n        2: Sigmoid(x)\n    \n    This class works as functional package of pytorch\n    \n    inputs:\n        code        :  the code for each activation (0,1,2)\n        input_tensor:  the input tensor\n        \n    returns:\n        result      :  the output of requested activation function with the same shape as input tensor\n    \n    \n    def __init__(self, code, input):\n        self.code = code\n        \n        self.result = torch.Tensor.new(input.shape)\n    \n    def Relu (self,input):\n        result = input - (input<0);\n        \n    def Tanh (self,input): #WARNING!!!\n        result = torch.tanh(input)\n        \n    def Sigmoid (self,input):\n        result = 1.0/(1 + torch.exp(-input))\n\n    \n    def choose_activation (self, code,input):\n        if self.code ==0:\n            self.result = self.Relu(input)\n        elif self.cod

# Network Class

In [118]:
class net:
    """
    The network class. It has the following methods:
        param      :  returns the parameter which is asked for. Not the data! The object... 
                        Data is accessible through object.data method)
        make_arch  :  makes the architecture of the network by taking a sequential list of [fc1,act1,fc2,act2,...]
    """
    
    def __init__ (self, seq=[]):
        self.param_list =[]
        self.act_list = []
        self.make_arch(seq)
        
    def param (self,layer):
        return self.param_list[layer]
    
    def make_arch(self,seq):
        seq_len = len (seq)
        for layer in range (0,seq_len,2):
            # we are sure that seq[layer] is a FC object -> need to know in / out
            dim_in, dim_out = seq[layer].input , seq[layer].output 
            self.param_list.append ( Weight(layer, dim_in, dim_out) )#object (Activation / Linear) )   #... should be filled wrt seq type and inputs
            # activation recognition
            if seq[layer+1]=='relu':
                self.act_list.append(0)
            elif seq[layer+1]=='tanh':
                self.act_list.append(1)
            elif seq[layer+1]=='sig':
                self.act_list.append(2)
            else: raise UnknownActivation
                
    def forward(self,x):
        
        for layer,W in enumerate(self.param_list):
            s=x.mm(W.data)
            x=Activation(self.act_list[layer],s)
            print(x)
        return x
        

# Draft, tests, and other stuff

In [119]:
seq = [Linear(4,7),'relu']    
model = net (seq)

In [120]:
model.forward(Tensor([[1.,5.,-1.,-1.]]))


 0.0000  4.7655  9.7860  0.5926  0.0000  1.8284  0.0000
[torch.FloatTensor of size 1x7]




 0.0000  4.7655  9.7860  0.5926  0.0000  1.8284  0.0000
[torch.FloatTensor of size 1x7]

In [110]:
a=Tensor([[1,3,-5,0,-2,1]])

In [107]:
b=(a>0)

In [109]:
b.float()


 1  1  0  0  0  1
[torch.FloatTensor of size 1x6]

In [112]:
(a<0).float()


 0  0  1  0  1  0
[torch.FloatTensor of size 1x6]